<a href="https://colab.research.google.com/github/Squarkk/Squarkk/blob/main/NFT_DOWNLOADER_factory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0 Test

## Testing total

In [ ]:
import pprint

def get_data(collection,download_images=False):
        import requests
        import os
        import json
        import math
        from tqdm import tqdm

        CollectionName = collection.lower()

        # Get information regarding collection

        collection = requests.get(f"http://api.opensea.io/api/v1/collection/{CollectionName}?format=json")

        print(collection)

        return collection



In [ ]:
import pprint
pprint.pprint(get_data("BOREDapeyachtclub").json())

## Testing random nft

In [16]:
def test_random_nft(collection):
        import json 
        import requests
        CollectionName = collection.lower()

        return json.loads(requests.get(f"https://api.opensea.io/api/v1/assets?order_direction=asc&offset=1&limit=50&collection={CollectionName}&format=json").content.decode())['assets'][12]



In [17]:
test_random_nft('CRYPTOPUNKS')

{'animation_original_url': None,
 'animation_url': None,
 'asset_contract': {'address': '0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb',
  'asset_contract_type': 'non-fungible',
  'buyer_fee_basis_points': 0,
  'created_date': '2018-01-23T04:51:38.832339',
  'default_to_fiat': False,
  'description': 'CryptoPunks launched as a fixed set of 10,000 items in mid-2017 and became one of the inspirations for the ERC-721 standard. They have been featured in places like The New York Times, Christie’s of London, Art|Basel Miami, and The PBS NewsHour.',
  'dev_buyer_fee_basis_points': 0,
  'dev_seller_fee_basis_points': 0,
  'external_link': 'https://www.larvalabs.com/cryptopunks',
  'image_url': 'https://lh3.googleusercontent.com/BdxvLseXcfl57BiuQcQYdJ64v-aI8din7WPk0Pgo3qQFhAUH-B6i-dCqqc_mCkRIzULmwzwecnohLhrcH8A9mpWIZqA7ygc52Sr81hE=s120',
  'name': 'CryptoPunks',
  'nft_version': 'unsupported',
  'only_proxied_transfers': False,
  'opensea_buyer_fee_basis_points': 0,
  'opensea_seller_fee_basis_po

# 1 Macro classe

In [5]:
#superclass dispatcher
class NFT_DATABASE:
  def __init__(self,factory):
    import requests
    import os
    import json
    import math
    from tqdm import tqdm
    
    print("hello")
    self.data=factory().get_data()
    self.info=factory().get_info()
    self.csv=factory().get_output()



# 2 Subclasse e subsub

## Cryptopunks

In [14]:
#Cryptopunk
class cryptopunks:
    def get_data(self,collection="CRYPTOPUNKS",download_images=False):
        import requests
        import os
        import json
        import math
        from tqdm import tqdm

        CollectionName = collection.lower()

        # Get information regarding collection

        collection = requests.get(f"http://api.opensea.io/api/v1/collection/{CollectionName}?format=json")

        if collection.status_code == 429:
            print("Server returned HTTP 429. Request was throttled. Please try again in about 5 minutes.")

        if collection.status_code == 404:
            print("NFT Collection not found.\n\n(Hint: Try changing the name of the collection in the Python script, line 11.)")
            exit()

        collectioninfo = json.loads(collection.content.decode())

        # Create image folder if it doesn't exist.

        if not os.path.exists('./images'):
            os.mkdir('./images')

        if not os.path.exists(f'./images/{CollectionName}'):
            os.mkdir(f'./images/{CollectionName}')

        if not os.path.exists(f'./images/{CollectionName}/data'):
            os.mkdir(f'./images/{CollectionName}/data')

        # Get total NFT count

        count = int(collectioninfo["collection"]["stats"]["count"])

        # Opensea limits to 50 assets per API request, so here we do the division and round up.

        iter = math.ceil(count / 50)

        print(f"\nBeginning download of \"{CollectionName}\" collection.\n")

        # Define variables for statistics

        stats = {
        "DownloadedData": 0,
        "AlreadyDownloadedData": 0,
        "DownloadedImages": 0,
        "AlreadyDownloadedImages": 0,
        "FailedImages": 0
        }

        # Iterate through every unit
        for i in tqdm(range(iter)):
            offset = i * 50
            data = json.loads(requests.get(f"https://api.opensea.io/api/v1/assets?order_direction=asc&offset={offset}&limit=50&collection={CollectionName}&format=json").content.decode())

            if "assets" in data:
                for asset in data["assets"]:
                    formatted_number = f"{int(asset['token_id']):04d}"

                    #print(f"\n#{formatted_number}:")

                    # Check if data for the NFT already exists, if it does, skip saving it
                    if os.path.exists(f'./images/{CollectionName}/data/{formatted_number}.json'):
                        #print(f"  Data  -> [\u2713] (Already Downloaded)")
                        stats["AlreadyDownloadedData"] += 1
                    else:
                        # Take the JSON from the URL, and dump it to the respective file.
                        dfile = open(f"./images/{CollectionName}/data/{formatted_number}.json", "w+")
                        json.dump(asset, dfile, indent=3)
                        dfile.close()
                        #print(f"  Data  -> [\u2713] (Successfully downloaded)")
                        stats["DownloadedData"] += 1

                    # Check if image already exists, if it does, skip saving it
                    if os.path.exists(f'./images/{CollectionName}/{formatted_number}.png') and download_images==True:
                        #print(f"  Image -> [\u2713] (Already Downloaded)")
                        stats["AlreadyDownloadedImages"] += 1
                    else:
                        # Make the request to the URL to get the image
                        if not asset["image_original_url"] == None and download_images==True:
                            image = requests.get(asset["image_original_url"])
                            # If the URL returns status code "200 Successful", save the image into the "images" folder.
                            if image.status_code == 200 and download_images==True:
                                file = open(f"./images/{CollectionName}/{formatted_number}.png", "wb+")
                                file.write(image.content)
                                file.close()
                                #print(f"  Image -> [\u2713] (Successfully downloaded)")
                                stats["DownloadedImages"] += 1
                        elif download_images==True:
                                image = requests.get(asset["image_url"])
                                # If the URL returns a status code other than "200 Successful", alert the user and don't save the image
                                if download_images==True:
                                    #print(f"  Image -> [!] (HTTP Status {image.status_code})")
                                    stats["FailedImages"] += 1
                                continue

                    
                    

        print(f"""
        Finished downloading collection.
        Statistics
        -=-=-=-=-=-
        Total of {count} units in collection "{CollectionName}".
        Downloads:
        JSON Files ->
            {stats["DownloadedData"]} successfully downloaded
            {stats["AlreadyDownloadedData"]} already downloaded
        Images ->
            {stats["DownloadedImages"]} successfully downloaded
            {stats["AlreadyDownloadedImages"]} already downloaded
            {stats["FailedImages"]} failed
        You can find the images (if downloaded) in the images/{CollectionName} folder.
        The JSON for each NFT can be found in the images/{CollectionName}/image_data folder.  
        
        
        
        """)

        #Defining function to extract data from the single json
        def extract_nft_info(path):
            import json
            response=json.load(open(path))
            #Last sale 
            if response['last_sale'] is None:
                price = None 
                symbol= None
                hashino= None
                hash=None
                last_trade_date= None
                value_at_date=None
                decimals = None
                hashlist=None
                symbol_price=None

            else:
                hash = response['last_sale']['total_price'].rjust(response['last_sale']['payment_token']['decimals']+1, '0')
                decimals=response['last_sale']['payment_token']['decimals']
                hashino= len(hash)
                hashlist = list(hash)
                hashlist.insert(len(hash)-response['last_sale']['payment_token']['decimals'], '.')
                hashlist=''.join(hashlist)
                price=round(float(hashlist),4)
                symbol = response['last_sale']['payment_token']['symbol']
                last_trade_date = response['last_sale']['event_timestamp'][0:10]
                value_at_date = round(float(price)*float(response ['last_sale']['payment_token']['usd_price']),2)
                symbol_price = round(float(response['last_sale']['payment_token']['usd_price']),4)

            #packing everything together
            data={'token_id': int(response['token_id']),
                'num_sales':int(response['num_sales']),
                'name':response['name'],
                'creation_date':response['asset_contract']['created_date'][0:10],
                'price':price,
                'symbol':symbol,
                'symbol_price':symbol_price,
                'last_trade_date':last_trade_date,
                'value_usd':value_at_date,
                'n_traits':len(response['traits'])}

            return data

        #Initializing output dataframe
        import pandas as pd
        output=pd.DataFrame()
        #Iterating for each json file with the extract_nft_info
        for i in  tqdm(os.listdir(f'./images/cryptopunks/data')):
            path='./images/cryptopunks/data/'+i
            chunk=pd.DataFrame([extract_nft_info(path)])
            output = output.append(chunk, ignore_index=True)

        
        
        return output

    def get_info(self):
        return 'info_here'
    
    def get_output(self):
        return 'output_here'
    


## Bored Ape

In [6]:
#Cryptopunk
class boredape:
    def get_data(self,collection="BOREDapeyachtclub",download_images=False):
        import requests
        import os
        import json
        import math
        from tqdm import tqdm

        CollectionName = collection.lower()

        # Get information regarding collection

        collection = requests.get(f"http://api.opensea.io/api/v1/collection/{CollectionName}?format=json")

        if collection.status_code == 429:
            print("Server returned HTTP 429. Request was throttled. Please try again in about 5 minutes.")

        if collection.status_code == 404:
            print("NFT Collection not found.\n\n(Hint: Try changing the name of the collection in the Python script, line 11.)")
            exit()

        collectioninfo = json.loads(collection.content.decode())

        # Create image folder if it doesn't exist.

        if not os.path.exists('./images'):
            os.mkdir('./images')

        if not os.path.exists(f'./images/{CollectionName}'):
            os.mkdir(f'./images/{CollectionName}')

        if not os.path.exists(f'./images/{CollectionName}/data'):
            os.mkdir(f'./images/{CollectionName}/data')

        # Get total NFT count

        count = int(collectioninfo["collection"]["stats"]["count"])

        # Opensea limits to 50 assets per API request, so here we do the division and round up.

        iter = math.ceil(count / 50)

        print(f"\nBeginning download of \"{CollectionName}\" collection.\n")

        # Define variables for statistics

        stats = {
        "DownloadedData": 0,
        "AlreadyDownloadedData": 0,
        "DownloadedImages": 0,
        "AlreadyDownloadedImages": 0,
        "FailedImages": 0
        }

        # Iterate through every unit
        for i in tqdm(range(iter)):
            offset = i * 50
            data = json.loads(requests.get(f"https://api.opensea.io/api/v1/assets?order_direction=asc&offset={offset}&limit=50&collection={CollectionName}&format=json").content.decode())

            if "assets" in data:
                for asset in data["assets"]:
                    formatted_number = f"{int(asset['token_id']):04d}"

                    #print(f"\n#{formatted_number}:")

                    # Check if data for the NFT already exists, if it does, skip saving it
                    if os.path.exists(f'./images/{CollectionName}/data/{formatted_number}.json'):
                        #print(f"  Data  -> [\u2713] (Already Downloaded)")
                        stats["AlreadyDownloadedData"] += 1
                    else:
                        # Take the JSON from the URL, and dump it to the respective file.
                        dfile = open(f"./images/{CollectionName}/data/{formatted_number}.json", "w+")
                        json.dump(asset, dfile, indent=3)
                        dfile.close()
                        #print(f"  Data  -> [\u2713] (Successfully downloaded)")
                        stats["DownloadedData"] += 1

                    # Check if image already exists, if it does, skip saving it
                    if os.path.exists(f'./images/{CollectionName}/{formatted_number}.png') and download_images==True:
                        #print(f"  Image -> [\u2713] (Already Downloaded)")
                        stats["AlreadyDownloadedImages"] += 1
                    else:
                        # Make the request to the URL to get the image
                        if not asset["image_original_url"] == None and download_images==True:
                            image = requests.get(asset["image_original_url"])
                            # If the URL returns status code "200 Successful", save the image into the "images" folder.
                            if image.status_code == 200 and download_images==True:
                                file = open(f"./images/{CollectionName}/{formatted_number}.png", "wb+")
                                file.write(image.content)
                                file.close()
                                #print(f"  Image -> [\u2713] (Successfully downloaded)")
                                stats["DownloadedImages"] += 1
                        elif download_images==True:
                                image = requests.get(asset["image_url"])
                                # If the URL returns a status code other than "200 Successful", alert the user and don't save the image
                                if download_images==True:
                                    #print(f"  Image -> [!] (HTTP Status {image.status_code})")
                                    stats["FailedImages"] += 1
                                continue

                    
                    

        print(f"""
        Finished downloading collection.
        Statistics
        -=-=-=-=-=-
        Total of {count} units in collection "{CollectionName}".
        Downloads:
        JSON Files ->
            {stats["DownloadedData"]} successfully downloaded
            {stats["AlreadyDownloadedData"]} already downloaded
        Images ->
            {stats["DownloadedImages"]} successfully downloaded
            {stats["AlreadyDownloadedImages"]} already downloaded
            {stats["FailedImages"]} failed
        You can find the images (if downloaded) in the images/{CollectionName} folder.
        The JSON for each NFT can be found in the images/{CollectionName}/image_data folder.  
        
        
        
        """)

        #Defining function to extract data from the single json
        def extract_nft_info(path):
            import json
            response=json.load(open(path))
            #Last sale 
            if response['last_sale'] is None:
                price = None 
                symbol= None
                hashino= None
                hash=None
                last_trade_date= None
                value_at_date=None
                decimals = None
                hashlist=None
                symbol_price=None

            else:
                hash = response['last_sale']['total_price'].rjust(response['last_sale']['payment_token']['decimals']+1, '0')
                decimals=response['last_sale']['payment_token']['decimals']
                hashino= len(hash)
                hashlist = list(hash)
                hashlist.insert(len(hash)-response['last_sale']['payment_token']['decimals'], '.')
                hashlist=''.join(hashlist)
                price=round(float(hashlist),4)
                symbol = response['last_sale']['payment_token']['symbol']
                last_trade_date = response['last_sale']['event_timestamp'][0:10]
                value_at_date = round(float(price)*float(response ['last_sale']['payment_token']['usd_price']),2)
                symbol_price = round(float(response['last_sale']['payment_token']['usd_price']),4)

            #packing everything together
            data={'token_id': int(response['token_id']),
                'num_sales':int(response['num_sales']),
                'name':response['name'],
                'creation_date':response['asset_contract']['created_date'][0:10],
                'price':price,
                'symbol':symbol,
                'symbol_price':symbol_price,
                'last_trade_date':last_trade_date,
                'value_usd':value_at_date,
                'n_traits':len(response['traits'])}

            return data

        #Initializing output dataframe
        import pandas as pd
        output=pd.DataFrame()
        #Iterating for each json file with the extract_nft_info
        for i in  tqdm(os.listdir(f'./images/{CollectionName}/data')):
            path=f'./images/{CollectionName}/data/'+i
            chunk=pd.DataFrame([extract_nft_info(path)])
            output = output.append(chunk, ignore_index=True)

        
        
        return output

    def get_info(self):
        return 'info_here'
    
    def get_output(self):
        return 'output_here'
    


# 3 Utilizzo

In [15]:
analisi = NFT_DATABASE(cryptopunks)


hello

Beginning download of "cryptopunks" collection.



100%|██████████| 200/200 [02:29<00:00,  1.34it/s]



        Finished downloading collection.
        Statistics
        -=-=-=-=-=-
        Total of 9999 units in collection "cryptopunks".
        Downloads:
        JSON Files ->
            0 successfully downloaded
            10000 already downloaded
        Images ->
            0 successfully downloaded
            0 already downloaded
            0 failed
        You can find the images (if downloaded) in the images/cryptopunks folder.
        The JSON for each NFT can be found in the images/cryptopunks/image_data folder.  
        
        
        
        


100%|██████████| 10000/10000 [01:10<00:00, 142.58it/s]


In [13]:
analisi.data


,token_id,num_sales,name,creation_date,price,symbol,symbol_price_at_date,last_trade_date,value_at_date_usd,n_traits
0,320,0,CryptoPunk #320,2018-01-23,NaN,None,NaN,None,NaN,3
1,3965,4,CryptoPunk #3965,2018-01-23,25.50,ETH,3824.22,2021-07-30,97517.61,3
2,2196,2,CryptoPunk #2196,2018-01-23,120.00,ETH,3824.22,2021-08-03,458906.40,4
3,2594,1,CryptoPunk #2594,2018-01-23,0.13,ETH,3824.22,2018-01-02,497.15,3
4,1670,0,CryptoPunk #1670,2018-01-23,NaN,None,NaN,None,NaN,4
...,...,...,...,...,...,...,...,...,...,...
9995,584,0,CryptoPunk #584,2018-01-23,NaN,None,NaN,None,NaN,4
9996,7595,1,CryptoPunk #7595,2018-01-23,0.30,ETH,3824.22,2017-07-03,1147.27,4
9997,8018,4,CryptoPunk #8018,2018-01-23,87.00,ETH,3824.22,2021-08-27,332707.14,3
9998,5179,3,CryptoPunk #5179,2018-01-23,2.49,ETH,3824.22,2020-09-20,9522.31,3
